In [1]:
import pandas as pd


clustering_algorithms = ['gaussian_mixture', 'kmeans']
fusion_types = ["add", "cat"]
weights = ["0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"]
iteration = 10
class_num = ["23", "34"]
conv_types = ["se", "re"]
eval_methods = ['ari', 'nmi', 'purity']

name_column = []
for c_num in class_num:
    data_name = "rico_" + c_num
    name_column.append(data_name)
    row_name = 'seq2seq_' + c_num
    name_column.append(row_name)
    for c_type in conv_types:
        row_name = 'conv_' + c_type + '_' + c_num
        name_column.append(row_name)
    for f_type in fusion_types:
        row_name = 'rico' + c_num + '_seq2seq' + c_num + '_' + f_type
        name_column.append(row_name)
    for f_type in fusion_types:
        row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type
        name_column.append(row_name)
    for c_type in conv_types:
        for f_type in fusion_types:
            row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
            name_column.append(row_name)
        for f_type in fusion_types:
            row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
            name_column.append(row_name)
    for f_type in fusion_types:
        for w in weights:
            row_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + f_type + "_" + w
            name_column.append(row_name)
    for f_type in fusion_types:
        for w in weights:
            row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type + "_" + w
            name_column.append(row_name)
    for c_type in conv_types:
        for w in weights:
            for f_type in fusion_types:
                row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w             
                name_column.append(row_name)
            for f_type in fusion_types:
                row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w
                name_column.append(row_name)

column_name = [str(x) for x in range(30)]
df_ari_gmm = pd.DataFrame(columns=column_name)
df_ari_gmm = df_ari_gmm.assign(name = name_column)
df_ari_gmm.set_index('name', inplace=True)
df_ari_km = pd.DataFrame(columns=column_name)
df_ari_km = df_ari_km.assign(name = name_column)
df_ari_km.set_index('name', inplace=True)

df_nmi_gmm = pd.DataFrame(columns=column_name)
df_nmi_gmm = df_ari_gmm.assign(name = name_column)
df_nmi_gmm.set_index('name', inplace=True)
df_nmi_km = pd.DataFrame(columns=column_name)
df_nmi_km = df_ari_km.assign(name = name_column)
df_nmi_km.set_index('name', inplace=True)

df_purity_gmm = pd.DataFrame(columns=column_name)
df_purity_gmm = df_ari_gmm.assign(name = name_column)
df_purity_gmm.set_index('name', inplace=True)
df_purity_km = pd.DataFrame(columns=column_name)
df_purity_km = df_ari_km.assign(name = name_column)
df_purity_km.set_index('name', inplace=True)

def run(data_name, iter, row_name):    
    lres = []
    for i, eval_method in enumerate(eval_methods):
        csv_path = 'csv/' + eval_method + '/' + data_name + ".csv"
        with open(csv_path, 'r') as fd:
            res = fd.read()
        lres = res.split('\n')
        if eval_method == 'purity':
            temp = lres[-2].split('Total,')[-1].split(',')
            df_purity_gmm.loc[row_name, str(iter)] = float(temp[0].strip('\%'))/100
            df_purity_km.loc[row_name, str(iter)] = float(temp[1].strip('\%'))/100
        elif eval_method == 'nmi':
            temp = lres[-2].split(',')
            df_nmi_gmm.loc[row_name, str(iter)] = float(temp[0])
            df_nmi_km.loc[row_name, str(iter)] = float(temp[1])
        else:
            temp = lres[-2].split(',')
            df_ari_gmm.loc[row_name, str(iter)] = float(temp[0])
            df_ari_km.loc[row_name, str(iter)] = float(temp[1])

for c_num in class_num:
    data_name = "rico_" + c_num
    run(data_name, 0, data_name)
    for _iter in range(iteration):
        data_name = "seq2seq_" + c_num + "_" + str(_iter)
        row_name = 'seq2seq_' + c_num
        run(data_name, _iter, row_name)
        for c_type in conv_types:
            data_name = "conv_" + c_type + "_" + c_num + "_" + str(_iter)
            row_name = 'conv_' + c_type + '_' + c_num
            run(data_name, _iter, row_name)
        for f_type in fusion_types:
            data_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + str(_iter) + "_" + f_type
            row_name = 'rico' + c_num + '_seq2seq' + c_num + '_' + f_type
            run(data_name, _iter, row_name)
        for f_type in fusion_types:
            data_name = "conv_re_" + c_num + "_" + str(_iter) + "_conv_se_" + c_num + "_" + str(_iter) + "_" + f_type
            row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type
            run(data_name, _iter, row_name)
        for c_type in conv_types:
            for f_type in fusion_types:
                data_name = ("rico_" + c_num + "_conv_" + c_type + "_" +
                                c_num + "_" + str(_iter) + "_" + f_type)
                row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
                run(data_name, _iter, row_name)
            for f_type in fusion_types:
                data_name = ("seq2seq_" + c_num + "_" + str(_iter) + "_conv_" + c_type + "_" +
                                c_num + "_" + str(_iter) + "_" + f_type)
                row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
                run(data_name, _iter, row_name)
        for f_type in fusion_types:
            for w in weights:
                data_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + str(_iter) + "_" + f_type + "_" + w
                row_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + f_type + "_" + w
                run(data_name, _iter, row_name)
        for f_type in fusion_types:
            for w in weights:
                data_name = "conv_re_" + c_num + "_" + str(_iter) + "_conv_se_" + c_num + "_" + str(_iter) + "_" + f_type + "_" + w
                row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type + "_" + w
                run(data_name, _iter, row_name)
        for c_type in conv_types:
            for w in weights:
                for f_type in fusion_types:
                    data_name = ("rico_" + c_num + "_conv_" + c_type + "_" +
                                    c_num + "_" + str(_iter) + "_" + f_type + "_" + w)
                    row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w             
                    run(data_name, _iter, row_name)
                for f_type in fusion_types:
                    data_name = ("seq2seq_" + c_num + "_" + str(_iter) + "_conv_" + c_type + "_" +
                                    c_num + "_" + str(_iter) + "_" + f_type + "_" + w)
                    row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w
                    run(data_name, _iter, row_name)


In [2]:
df_ari_gmm['Mean'] = df_ari_gmm.mean(axis=1)
df_ari_gmm['Std'] = df_ari_gmm.std(axis=1)
df_ari_km['Mean'] = df_ari_km.mean(axis=1)
df_ari_km['Std'] = df_ari_km.std(axis=1)
df_nmi_gmm['Mean'] = df_nmi_gmm.mean(axis=1)
df_nmi_gmm['Std'] = df_nmi_gmm.std(axis=1)
df_nmi_km['Mean'] = df_nmi_km.mean(axis=1)
df_nmi_km['Std'] = df_nmi_km.std(axis=1)
df_purity_gmm['Mean'] = df_purity_gmm.mean(axis=1)
df_purity_gmm['Std'] = df_purity_gmm.std(axis=1)
df_purity_km['Mean'] = df_purity_km.mean(axis=1)
df_purity_km['Std'] = df_purity_km.std(axis=1)

In [3]:
df_purity_km

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,Mean,Std
name,,,,,,,,,,,,,,,,,,,,,
rico_23,0.343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3430,0.000000
seq2seq_23,0.336,0.309,0.36,0.417,0.377,0.309,0.377,0.357,0.324,0.377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3543,0.032976
conv_se_23,0.427,0.43,0.326,0.294,0.393,0.439,0.361,0.441,0.43,0.446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3987,0.051100
conv_re_23,0.279,0.249,0.246,0.227,0.274,0.244,0.2,0.31,0.21,0.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2519,0.032501
rico23_seq2seq23_add,0.334,0.377,0.393,0.36,0.377,0.259,0.39,0.356,0.426,0.404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3676,0.043789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
seq2seq_34_conv_re_34_cat_0.8,0.333,0.229,0.204,0.283,0.236,0.286,0.272,0.289,0.228,0.331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2691,0.041748
rico_34_conv_re_34_add_0.9,0.268,0.218,0.307,0.305,0.299,0.236,0.282,0.235,0.289,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2689,0.030687
rico_34_conv_re_34_cat_0.9,0.324,0.26,0.274,0.294,0.317,0.252,0.272,0.274,0.264,0.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2791,0.023386


In [4]:
df_ari_gmm.to_csv('../result/ari_gmm_new.csv')
df_ari_km.to_csv('../result/ari_km_new.csv')
df_nmi_gmm.to_csv('../result/nmi_gmm_new.csv')
df_nmi_km.to_csv('../result/nmi_km_new.csv')
df_purity_gmm.to_csv('../result/purity_gmm_new.csv')
df_purity_km.to_csv('../result/purity_km_new.csv')
